In [1]:
import pandas as pd
import numpy as np
from numpy import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import re
import datetime
from math import sqrt
from __future__ import division
from enum import Enum
import math

In [2]:
movies = pd.read_csv("Dataset/ml-latest-small/movies.csv")
ratings = pd.read_csv("Dataset/ml-latest-small/ratings.csv")

In [3]:
ratings = ratings.drop(['timestamp'],axis=1)
movies = movies.drop(['genres'],axis=1)
overall_ratings = ratings.groupby(['movieId'],as_index=False).agg({"rating": np.mean, "userId": pd.Series.nunique})
overall_ratings = overall_ratings.rename(columns={"rating": "total_avg_rating","userId": "total_num_rating"})
movies = pd.merge(movies,overall_ratings,on=['movieId'])

In [4]:
col = list(ratings.columns)
x = col[:]
x.remove('rating')
y = 'rating'

In [15]:
split = np.random.rand(len(ratings)) < 0.85
trainDf = ratings[split]
testDf = ratings[~split]
print trainDf.shape
print testDf.shape

(85024, 3)
(14980, 3)


In [16]:
print datetime.datetime.now()
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(trainDf[x],trainDf[y])
print datetime.datetime.now()

2017-10-03 07:33:17.024446
2017-10-03 07:33:17.140075


In [18]:
print datetime.datetime.now()
train_result = knn.predict(trainDf[x])
test_result = knn.predict(testDf[x])
print datetime.datetime.now()

2017-10-03 07:35:05.902541
2017-10-03 07:35:06.215674


In [19]:
testDf = testDf.reset_index()
testDf["knn_rating"] = pd.Series(test_result,index=testDf.index)
sqrt(mean_squared_error(testDf.rating, testDf.knn_rating))

1.0825244318385474

In [20]:
testDf["actual_rat_class"] = 0
testDf["pred_rat_class"] = 0

for idx in range(0,len(testDf)):
    actual_rating = testDf.loc[idx,"rating"]
    if actual_rating <= 2.0:
        testDf.loc[idx,"actual_rat_class"] = 1
    elif actual_rating <= 3.5:
        testDf.loc[idx,"actual_rat_class"] = 2
    else:
        testDf.loc[idx,"actual_rat_class"] = 3
    
    pred_rating = testDf.loc[idx,"knn_rating"]
    if pred_rating <= 2.0:
        testDf.loc[idx,"pred_rat_class"] = 1
    elif pred_rating <= 3.5:
        testDf.loc[idx,"pred_rat_class"] = 2
    else:
        testDf.loc[idx,"pred_rat_class"] = 3
        
print sqrt(mean_squared_error(testDf.actual_rat_class, testDf.pred_rat_class))

0.812412057478


In [21]:
testDf["actual_rat_class"] = 0
testDf["pred_rat_class"] = 0

for idx in range(0,len(testDf)):
    actual_rating = testDf.loc[idx,"rating"]
    if actual_rating < 3.5:
        testDf.loc[idx,"actual_rat_class"] = 1
    else:
        testDf.loc[idx,"actual_rat_class"] = 2
    
    pred_rating = testDf.loc[idx,"knn_rating"]
    if pred_rating < 3.5:
        testDf.loc[idx,"pred_rat_class"] = 1
    else:
        testDf.loc[idx,"pred_rat_class"] = 2

sqrt(mean_squared_error(testDf.actual_rat_class, testDf.pred_rat_class))

0.635404058656387

In [5]:
split = np.random.rand(len(ratings)) < 0.85
trainDf = ratings[split]
testDf = ratings[~split]
print trainDf.shape
print testDf.shape

(85119, 3)
(14885, 3)


In [6]:
print datetime.datetime.now()

userIdList = list(ratings.userId.unique())
#userIdList = userIdList[:5]
simUsersList = []
simUserMov = []
corrMat = np.empty(shape=(len(userIdList)+1,len(userIdList)+1))
corrMat[:,:] = None

def getNormalDist(arr):
    std = np.std(arr)
    mean = np.mean(arr)
    if std != 0:
        for elem in arr:
            elem = (elem - mean)/std
    return arr
    

for user in userIdList:
    userMovRat = trainDf[trainDf['userId'] == user]
    userMov = set(userMovRat['movieId'])
    simUsersList = [i for i in userIdList if i != user]
    for simUser in simUsersList:
        if np.isnan(corrMat[user][simUser]):
            simUserRat = trainDf[trainDf['userId'] == simUser]
            simUserMov = set(simUserRat['movieId'])
            simMovies = userMov.intersection(simUserMov)
            simMovList = list(simMovies)
            if len(simMovList) > 1:
                userComMov = userMovRat.loc[userMovRat.movieId.isin(simMovList)]
                simUserComMov = simUserRat.loc[simUserRat.movieId.isin(simMovList)]
                userComMov = userComMov.sort_values("movieId",ascending=True)
                simUserComMov = simUserComMov.sort_values("movieId",ascending=True)
                corrMat[user,simUser] = pearsonr(getNormalDist(list(userComMov.rating)), getNormalDist(list(simUserComMov.rating)))[0]
                corrMat[simUser,user] = corrMat[user,simUser]
            else:
                corrMat[user,simUser] = 0
                
                
for i in range(1,len(corrMat)):
    corrMat[i,i] = 1
corrMat[np.isnan(corrMat)] = 0
                
print datetime.datetime.now()

2017-10-03 20:18:01.075244


/Users/suribabu/Projects/Anaconda/lib/python2.7/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


2017-10-03 20:30:52.706791


In [7]:
print datetime.datetime.now()

testDf = testDf.reset_index(drop=True)
testDf["pred_rating"] = 0.00
for i in range(len(testDf)):
    user_id = testDf.loc[i,"userId"]
    movie_id = testDf.loc[i,"movieId"]
    user_similarity = corrMat[user_id,:]
    sim_rating_prod = 0.0
    total_user_sim = 0.0
    for curr_user_id,curr_user_rat in enumerate(user_similarity):
        if curr_user_rat >= 0.4:
            df = trainDf[(trainDf["userId"] == curr_user_id) & (trainDf["movieId"] == movie_id)]
            if df.size > 0:
                sim_rating_prod = sim_rating_prod + (curr_user_rat * float(df.rating))
                total_user_sim = total_user_sim + curr_user_rat
    if total_user_sim > 0:
        pred_rating = sim_rating_prod / total_user_sim
        testDf.loc[i,"pred_rating"] = pred_rating
    
print datetime.datetime.now()

2017-10-03 20:31:12.116899
2017-10-03 21:24:08.229646


In [8]:
sqrt(mean_squared_error(testDf.rating, testDf.pred_rating))

1.8307220962233783

In [9]:
testDf["actual_rat_class"] = 0
testDf["pred_rat_class"] = 0

for idx in range(0,len(testDf)):
    actual_rating = testDf.loc[idx,"rating"]
    if actual_rating <= 2.0:
        testDf.loc[idx,"actual_rat_class"] = 1
    elif actual_rating <= 3.5:
        testDf.loc[idx,"actual_rat_class"] = 2
    else:
        testDf.loc[idx,"actual_rat_class"] = 3
    
    pred_rating = testDf.loc[idx,"pred_rating"]
    if pred_rating <= 2.0:
        testDf.loc[idx,"pred_rat_class"] = 1
    elif pred_rating <= 3.5:
        testDf.loc[idx,"pred_rat_class"] = 2
    else:
        testDf.loc[idx,"pred_rat_class"] = 3
        
sqrt(mean_squared_error(testDf.actual_rat_class, testDf.pred_rat_class))

0.9528346408858249

In [10]:
testDf["actual_rat_class"] = 0
testDf["pred_rat_class"] = 0

for idx in range(0,len(testDf)):
    actual_rating = testDf.loc[idx,"rating"]
    if actual_rating < 3.5:
        testDf.loc[idx,"actual_rat_class"] = 1
    else:
        testDf.loc[idx,"actual_rat_class"] = 2
    
    pred_rating = testDf.loc[idx,"pred_rating"]
    if pred_rating < 3.5:
        testDf.loc[idx,"pred_rat_class"] = 1
    else:
        testDf.loc[idx,"pred_rat_class"] = 2

sqrt(mean_squared_error(testDf.actual_rat_class, testDf.pred_rat_class))

0.6134753721603082

In [40]:
while True:
    user_id = input('Enter your input:')
    if user_id < 0:
        break
    else:
        similar_users = corrMat[user_id,:]
        movies_list = set()
        filtered_user_sim = [(idx,i) for idx,i in enumerate(similar_users) if i >= 0.4]

        for curr_user_rat in filtered_user_sim:
            movies_list = movies_list.union(set(ratings[ratings["userId"] == curr_user_rat[0]]["movieId"]))
        watched_movies = set(ratings[ratings["userId"] == user_id]["movieId"])
        movies_list = movies_list.difference(watched_movies)

        user_sim_df = {}
        user_sim_df["userId"] = [i[0] for i in filtered_user_sim]
        user_sim_df["similarity"] = [i[1] for i in filtered_user_sim]
        user_sim_df = pd.DataFrame(user_sim_df)

        recommend_movies = ratings[ratings["movieId"].isin(list(movies_list))]
        recommend_movies = pd.merge(recommend_movies,user_sim_df,on=["userId"])
        recommend_movies = recommend_movies.reset_index(drop=True)
        recommend_movies["sim_rat_prod"] = 0
        recommend_movies["sim_rat_prod"] = recommend_movies.apply(lambda x: x["similarity"] * x["rating"],axis=1)
        recommend_movies = recommend_movies.groupby(["movieId"],as_index=False)['sim_rat_prod'].agg(['mean','count']).reset_index()
        recommend_movies = recommend_movies.rename(index=str, columns={"mean": "mean_sim_rat_prod","count":"count_sim_rat"})
        recommended_movies_list = list(recommend_movies[recommend_movies['count_sim_rat'] >= 5].sort_values('mean_sim_rat_prod',ascending = False).head(5)['movieId'])

        for movieId in recommended_movies_list:
            print str(movies[movies["movieId"] == movieId]["title"])


Enter your input:4
7353    Fantastic Mr. Fox (2009)
Name: title, dtype: object
2764    Modern Times (1936)
Name: title, dtype: object
1034    Raise the Red Lantern (Da hong deng long gao g...
Name: title, dtype: object
802    Big Night (1996)
Name: title, dtype: object
4355    Talk to Her (Hable con Ella) (2002)
Name: title, dtype: object
Enter your input:-9
